# NOPT042 Constraint programming: Tutorial 4 – Search strategies

In [1]:
%load_ext ipicat

<IPython.core.display.Javascript object>

Picat version 3.2#8


### From last week:

* Solution to the Coin grid problem.
* Best model and solver for the problem? MIP, naturally expressed as an integer program
* Unsatisfiable instances - LP works well.
* For sparse solution sets heuristic approaches may be slow.

## Example: N-queens

Place $n$ queens on an $n\times n$ board so that none attack another. How to choose the decision variables?

* How large is the search space?
* Can we use symmetry breaking?
* Consider the _dual_ model.

In [8]:
!time picat queens/queens1.pi 32

Q...............................
..Q.............................
....Q...........................
.Q..............................
...Q............................
........Q.......................
..........Q.....................
............Q...................
..............Q.................
.....Q..........................
.................Q..............
.......................Q........
.........................Q......
.............................Q..
........................Q.......
..............................Q.
...........................Q....
...............................Q
..........................Q.....
............................Q...
...............Q................
..................Q.............
.........Q......................
.......Q........................
................Q...............
...........Q....................
....................Q...........
......Q.........................
.............Q..................
......................Q.........
..........

In [9]:
!cat queens/queens1.pi

% n-queens, first model
import cp.

main([N]) =>
    N := to_int(N),
    queens(N, Q),
    solve(Q),
    output(Q).


queens(N, Q) =>
    Q = new_array(N),
    Q :: 1..N,
    all_different(Q),
    all_different([$Q[I] - I : I in 1..N]),
    all_different([$Q[I] + I : I in 1..N]).


output(Q) =>
    N = Q.length,
    foreach(I in 1..N)
        foreach (J in 1..N)
            if Q[I] = J then
                print("Q")
            else
                print(".")
            end
        end,
        print("\n")
    end.


In [10]:
!time picat queens/queens2.pi 32

...............................Q
................Q...............
............Q...................
....Q...........................
..........................Q.....
.............Q..................
Q...............................
......Q.........................
............................Q...
.........................Q......
..Q.............................
..............................Q.
...............Q................
.........Q......................
.......................Q........
....................Q...........
......................Q.........
...Q............................
.............................Q..
..............Q.................
........Q.......................
.....................Q..........
.Q..............................
...........................Q....
.....Q..........................
.......Q........................
.................Q..............
........................Q.......
...........Q....................
...................Q............
..........

## Search strategies
And other solver options: see [Picat guide](http://picat-lang.org/download/picat_guide.pdf) (Section 12.6) and the [book](http://picat-lang.org/picatbook2015/constraint_solving_and_planning_with_picat.pdf) (Section 3.5)

In [2]:
%%picat -n queens
import cp. %try sat, try also mip with the other model

queens(N, Q) =>
    Q = new_array(N),
    Q :: 1..N,
    all_different(Q),
    all_different([$Q[I] - I : I in 1..N]),
    all_different([$Q[I] + I : I in 1..N]).

In [3]:
%%picat
main =>
    N = 32,
    queens(N, Q),
    time2(solve(Q)).


CPU time 8.111 seconds. Backtracks: 11461548




In [4]:
%%picat
main => println("here").

here



Which search strategy could work well for our model?

Here's how we can test multiple search strategies (code adapted from the book):

In [14]:
%%picat

% Variable selection
selection(VarSels) =>
    VarSels = [backward,constr,degree,ff,ffc,ffd,forward,inout,leftmost,max,min,up].
% Value selection
choice(ValSels) =>
    ValSels = [down,reverse_split,split,up,updown].

main =>
    selection(VarSels),
    choice(ValSels),
    Timeout = 1000, % Timeout in milliseconds
    %Timeout = 10000, % Timeout in milliseconds
    Ns = [32, 64, 128],
    
    foreach (N in Ns, VarSel in VarSels, ValSel in ValSels)
        queens(N,Q),
        time2(time_out(solve([VarSel,ValSel], Q),Timeout,Status)),
        println([N,VarSel,ValSel,Status])
    end.


CPU time 0.328 seconds. Backtracks: 350882

[32,backward,down,time_out]

CPU time 0.24 seconds. Backtracks: 0

[32,backward,reverse_split,time_out]

CPU time 0.468 seconds. Backtracks: 0

[32,backward,split,time_out]

CPU time 0.459 seconds. Backtracks: 186524

[32,backward,up,time_out]

CPU time 0.098 seconds. Backtracks: 197290

[32,backward,updown,success]

CPU time 0.35 seconds. Backtracks: 351975

[32,constr,down,time_out]

CPU time 0.377 seconds. Backtracks: 0

[32,constr,reverse_split,time_out]

CPU time 0.27 seconds. Backtracks: 0

[32,constr,split,time_out]

CPU time 0.23 seconds. Backtracks: 228589

[32,constr,up,time_out]

CPU time 0.119 seconds. Backtracks: 197290

[32,constr,updown,success]

CPU time 0.33 seconds. Backtracks: 336908

[32,degree,down,time_out]

CPU time 0.267 seconds. Backtracks: 0

[32,degree,reverse_split,time_out]

CPU time 0.24 seconds. Backtracks: 0

[32,degree,split,time_out]

CPU time 0.33 seconds. Backtracks: 206700

[32,degree,up,time_out]

CPU ti

## Exercises

### Exercise: Magic square
Arrange numbers $1,2,\dots,n^2$ in a square such that every row, every column, and the two main diagonals all sum to the same quantity. How many magic squares are there for a given $n$? Allow also for a partially filled instance. What is the best search strategy?

### Exercise: Minesweeper
Identify the positions of all mines in a given board. Try the following instance (from [the book](http://picat-lang.org/picatbook2015/constraint_solving_and_planning_with_picat.pdf)):
```
Instance = {
    {_,_,2,_,3,_},
    {2,_,_,_,_,_},
    {_,_,2,4,_,3},
    {1,_,3,4,_,_},
    {_,_,_,_,_,3},
    {_,3,_,3,_,_}
}.
```

### Exercise: Graph-coloring
1. Write a program that solves the (directed) graph 3-coloring problem with a given number of colors and a given graph. The graph is given by a list of edges, each edge is a 2-element list. We assume that vertices of the graph are $1,\dots,n$ where $n$ is the maximum number appearing in the list. 
2. Generalize your program to graph $k$-coloring where $k$ is a positive integer given on the input.
3. Modify your program to accept the incidence matrix (a 2D array) instead of the list of edges.
4. Add the flag `-n` to output the minimum number of colors (the chromatic number) of a given graph.
For example:
```
picat graph-coloring.pi [[1,2],[2,3],[3,4],[4,1]]
picat graph-coloring.pi [[1,2],[2,3],[3,1]] 4
picat graph-coloring.pi "{{0,1,1},{1,0,1},{1,1,0}}" 4
picat graph-coloring.pi -n [[1,2],[2,3],[3,4],[4,1]]
```

### Homework: TBA

In the _boardomino_ puzzle, the goal is to cover an $n\times n$ chess board with some fields missing by domino tiles, if it is possible. The input is given by a positive integer $n$ (the size of the board) and a list of pairs of missing fileds. The first line of the output should be `yes` or `no`. If a covering exists, some reasonble representation of it should be output as well.

1. Write a picat program for this puzzle. Here are some sample instances (the first one is unsatisfiable):
```
picat boardomino.pi 4 [[1,1],[4,4]]
picat boardomino.pi 8 [[1,1],[1,2]]
```
2. Try various solvers and solver settings (and you can try different models as well) and choose the best option based on the performance on the unsatisfiable instances of the form `n [1,1] [n,n]`.